In [2]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_rows', 100)
pd.set_option('display.float_format',lambda x : '%.2f' % x)
import numpy as np
np.set_printoptions(suppress=True)
from tqdm.notebook import tqdm

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import gc

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score

import lightgbm as lgb

In [2]:
!ls -R raw_data/

'ls' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


In [4]:
t_base_info = "./dataset/train/base_info.csv"
t_annual_report_info = "./dataset/train/annual_report_info.csv"
t_tax_info = "./dataset/train/tax_info.csv"
t_change_info = "./dataset/train/change_info.csv"
t_news_info = "./dataset/train/news_info.csv"
t_other_info = "./dataset/train/other_info.csv"
t_entprise_info = "./dataset/train/entprise_info.csv" # train labeled data

In [4]:
df_label_info = pd.read_csv(t_entprise_info, header=0)
print(df_label_info.head())

                                                 id  label
0  59b38c56de3836831ff90a77d892a13523b7494f6ed09ff7      1
1  da8691b210adb3f6be8064e006f220070565db287275ad38      0
2  82750f1b9d122350918121f97c99bf96e11aa24ee91504a9      0
3  f000950527a6feb6b2c6de6f85c1e7438ba5590be931e2ec      0
4  f1c1045b13d1832927e3743e49d2917f2d98424f0849a373      0


In [5]:
df_other_info = pd.read_csv(t_other_info, header=0)
print(df_other_info.head())
print(df_other_info.dtypes)
df_other_info.select_dtypes(['float64']).describe().T

                                                 id  legal_judgment_num  \
0  f000950527a6feb6d340f91da09e61347d8200cd2f0d1602                4.00   
1  f000950527a6feb608dd9322b74a99f60851207f36a3c94c                1.00   
2  d8071a739aa75a3b9f23966f8dae78fd226c272515b9c255                2.00   
3  216bd2aaf4d079242209b1496f81a36c7abed9dd0bb65ed3                 nan   
4  e9f7b28ec10e0470de9631c789f49acdd4e7cf9ed6db094b                 nan   

   brand_num  patent_num  
0        nan         nan  
1        nan         nan  
2        nan         nan  
3       1.00         nan  
4       2.00         nan  
id                     object
legal_judgment_num    float64
brand_num             float64
patent_num            float64
dtype: object


,count,mean,std,min,25%,50%,75%,max
legal_judgment_num,1006.00,14.80,54.58,1.00,1.00,3.00,9.00,959.00
brand_num,909.00,14.17,49.16,1.00,1.00,2.00,6.00,441.00
patent_num,396.00,47.01,97.75,1.00,4.00,16.00,50.00,1087.00


In [6]:
# for col in tqdm(df_other_info.select_dtypes(['float64']).columns):
#     df_other_info[col] = df_other_info[col].fillna(df_other_info[col].median())
# print(df_other_info.head())

In [7]:
training_set = pd.merge(df_label_info, df_other_info, how='left', on='id')
training_set = training_set.groupby(['id'], as_index=False).mean()
print(training_set.shape)

(14865, 5)


In [8]:
df_news_info = pd.read_csv(t_news_info, header=0)
print(df_news_info.head())
dict_atitude = {"积极": 1, "中立": 0, "消极": -1}
df_news_info['positive_negtive'] = df_news_info['positive_negtive'].map(lambda x : dict_atitude[x])
# public date 转变成迄今为止发生时间
cmp_date = "2020-10-09"
# df_news_info['public_date'] = df_news_info['public_date'].map(lambda x: check_date(x))
# df_news_info['public_date'].value_counts()

# 处理"xx小时前" 数据统一为昨天更新 
def handle_public_date(str):
    if(check_date(str) is False):
        return 1
    else:
        return days(cmp_date, str)
df_news_info['public_date'] = df_news_info['public_date'].map(lambda x: handle_public_date(x))

# 新闻发布时间，越久远 因子越小
df_news_info['news_factor'] = 1 - (df_news_info['public_date'] / df_news_info['public_date'].max())
df_news_info['news_factor'] = df_news_info[['news_factor', 'positive_negtive']].apply(lambda x: x['news_factor'] * x['positive_negtive'], axis=1)

# del df_news_info['positive_negtive']
# del df_news_info['public_date']
print(df_news_info.head())
df_news_info = df_news_info.groupby(['id'], as_index=False).sum()

                                                 id positive_negtive  \
0  f000950527a6feb62669d6a175fe6fdccd1eb4f7ca8e5016               积极   
1  f000950527a6feb6e8bd9919e2ca363359bcfa997a0f9de7               中立   
2  f000950527a6feb6e8bd9919e2ca363359bcfa997a0f9de7               消极   
3  d8071a739aa75a3bcf6fb0041ee883243251d30025ab9d45               中立   
4  f000950527a6feb6d71de3382afa0bc5ff87bb65477f698a               积极   

  public_date  
0  2016-12-30  
1  2017-08-09  
2  2016-02-29  
3  2018-06-08  
4  2015-06-29  


NameError: name 'check_date' is not defined

In [90]:
df_change_info = pd.read_csv(t_change_info, header=0)
del df_change_info['bgxmdm']
del df_change_info['bgq']
del df_change_info['bgh']
# del df_change_info['bgrq']
# print(df_change_info.head())
# print(df_change_info.shape)

df_change_info['bgcnt'] = 1
df_tmp = df_change_info[['id', 'bgcnt']]
df_tmp = df_tmp.groupby(['id'], as_index=False).sum()
print(df_tmp)

def get_last_time(x):
    return x.sort_values(ascending = False)[:1]
df_bgrq = df_change_info['bgrq'].groupby(df_change_info['id']).apply(get_last_time)

df_change_info = pd.merge(df_tmp, df_bgrq, how='left', on='id')
df_change_info['bgrq'] = df_change_info['bgrq'].apply(lambda x: str(x)[:4])

print(df_change_info.shape)
print(df_change_info.head())

# df_change_info = df_change_info.groupby(['id'], as_index=False).sum()


                                                    id  bgcnt
0     09912c34159b1720558a419983a989f1dd2e0ed69a044ca3      9
1     0ba26b418aa50d3c35e3d01d414b2c9f07efa6997fbf4394      5
2     175ebe5f059ec05007223e9af0a48b885f4cbfa833d93eed     11
3     175ebe5f059ec05036d901021be6da41057ae3ee1fe6b8bb     15
4     175ebe5f059ec0503eb86987a7b6da7c40c6e69d5b54736f      2
...                                                ...    ...
8721  f1c1045b13d18329fff6b6c261a696a54cc111b991a55a72      1
8722  f6eb47aa168d4141ebdc3e35a7b37a3cb15baa8608f730a0      2
8723  ff2c78b9022767a53da2e5a811e4aa9583e32efc9899f9f3      2
8724  ff2c78b9022767a5439dae566c7603a03a6178241dacf5c9     13
8725  ff2c78b9022767a583c0800a3f450666162ea0e4566cfd19     22

[8726 rows x 2 columns]
(8726, 3)
                                                 id  bgcnt  bgrq
0  09912c34159b1720558a419983a989f1dd2e0ed69a044ca3      9  2019
1  0ba26b418aa50d3c35e3d01d414b2c9f07efa6997fbf4394      5  2019
2  175ebe5f059ec05007223e9

In [20]:
# 数据集7：entprise_info.csv
# 带标注的企业数据。每一行代表一个企业，每一行2列，其中id列为企业唯一标识，label列为标注（1：有非法集资风险，0：无非法集资风险），列之间采用“,”分隔符分割。

# 训练集 id 及标签

entprise_info = pd.read_csv(t_entprise_info)

print(entprise_info.shape)
entprise_info.head()

(14865, 2)


,id,label
0,59b38c56de3836831ff90a77d892a13523b7494f6ed09ff7,1
1,da8691b210adb3f6be8064e006f220070565db287275ad38,0
2,82750f1b9d122350918121f97c99bf96e11aa24ee91504a9,0
3,f000950527a6feb6b2c6de6f85c1e7438ba5590be931e2ec,0
4,f1c1045b13d1832927e3743e49d2917f2d98424f0849a373,0


In [21]:
# 数据集8（验证集）：entprise_evaluate.csv
# 未标注企业数据。参赛队伍需提交的最终结果数据集，每一行代表一个企业，每一行有 2 列, 其中id列为企业唯一标识，score列为空，列之间采用“,”分隔符分割。

# 测试集 id

entprise_evaluate = pd.read_csv('./dataset/entprise_evaluate.csv')

print(entprise_evaluate.shape)
entprise_evaluate.head()

(10000, 2)


,id,score
0,82750f1b9d1223508ee329d47e27d35176c93eb9f35e9c1a,nan
1,f000950527a6feb670cc1c87c2025f3922aaa4a0206a0a33,nan
2,e9f7b28ec10e04700ef4db75a494f9a1e8e8b09555e6afa1,nan
3,beb4aaaa89e0a0ae9d77bd5d7665be6342f552f51840cf19,nan
4,e9f7b28ec10e0470ee4172cec0133b6826c34f27d3dff204,nan


In [23]:
# 数据集1：base_info.csv
# 包含数据集7和8中涉及到的所有企业的基本信息，每一行代表一个企业的基本数据，每一行有33列，其中id列为企业唯一标识，列之间采用“,”分隔符分割。
# 数据格式如下：
# [id:企业唯一标识, oplocdistrict:行政区划代码, industryphy:行业类别代码, industryco:行业细类代码, dom:经营地址, opscope:经营范围, enttype:企业类型, enttypeitem:企业类型小类, opfrom:经营期限起, opto:经营期限止, state:状态, orgid:机构标识, jobid:职位标识, adbusign:是否广告经营, townsign:是否城镇, regtype:主题登记类型, empnum:从业人数, compform:组织形式, parnum:合伙人数, exenum:执行人数, opform:经营方式, ptbusscope:兼营范围, venind:风险行业, enttypeminu:企业类型细类, midpreindcode:中西部优势产业代码, protype:项目类型, oploc:经营场所, regcap:注册资本（金）, reccap:实缴资本, forreccap:实缴资本（外方）, forregcap:注册资本（外方）, congro:投资总额, enttypegb:企业（机构）类型]

# 基础信息表

base_info = pd.read_csv(t_base_info)

print(base_info.shape)
base_info.head()

(24865, 33)


,id,oplocdistrict,industryphy,industryco,dom,opscope,enttype,enttypeitem,opfrom,opto,state,orgid,jobid,adbusign,townsign,regtype,empnum,compform,parnum,exenum,opform,ptbusscope,venind,enttypeminu,midpreindcode,protype,oploc,regcap,reccap,forreccap,forregcap,congro,enttypegb
0,47645761dc56bb8c5fae00114b768b5d9b6e917c3aec07c4,340223,M,7513.00,31487d8f256f16bd6244b7251be2ebb24d1db51663c654...,纳米新材料、机械设备、五金配件加工、销售及技术推广服务，道路货物运输。（依法须经批准的项目，...,1100,1150.00,2019-07-11 00:00:00,NaN,6,340223010010000000,340200000000115392,0,0,1,5.00,nan,nan,nan,NaN,nan,nan,1151.00,nan,nan,2367b4cac96d8598,50.00,nan,nan,nan,nan,1151
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,340222,O,8090.00,31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3...,健身服务。（依法须经批准的项目，经相关部门批准后方可开展经营活动）,9600,nan,2017-09-06,NaN,6,340222060010000000,340200000000112114,0,1,1,3.00,1.00,nan,nan,10,nan,3.00,nan,nan,nan,31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3...,10.00,nan,nan,nan,nan,9600
2,59b38c56de3836838082cfcb1a298951abfe15e6940c49ba,340202,R,9053.00,31487d8f256f16bd6244b7251be2ebb2ae36cd652943e8...,文化娱乐经纪人服务；境内文艺活动组织与策划；文化艺术交流活动组织策划；演出经纪；其他文化艺术...,1100,1150.00,2020-09-14 14:46:30,NaN,6,340202010010000000,400000000000753910,0,0,1,2.00,nan,1.00,nan,NaN,nan,nan,1151.00,nan,nan,2367b4cac96d8598,100.00,nan,nan,nan,nan,1151
3,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,340221,L,7212.00,746df9aaed8578571760c563abe882c8ba25209fc6d5db...,投资管理及咨询(证券、期货除外)；企业管理。（依法须经批准的项目，经相关部门批准后方可开展经...,4500,4540.00,2015-09-30,NaN,6,340221010010000000,400000000000013538,0,1,1,2.00,nan,nan,nan,01-以个人财产出资,nan,nan,nan,nan,nan,2367b4cac96d8598,10.00,nan,nan,nan,nan,4540
4,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,340202,R,8810.00,31487d8f256f16bd6244b7251be2ebb2ae36cd652943e8...,境内文化艺术交流活动策划；企业形象策划；礼仪庆典服务；翻译服务；专利代理；广告设计、制作、代...,1100,1130.00,2017-12-01,2067-11-30,7,340200000000000000,400000000000283237,0,0,1,nan,nan,nan,nan,NaN,nan,nan,nan,nan,nan,2367b4cac96d8598,100.00,nan,nan,nan,nan,1130


In [24]:
for col in tqdm(base_info.columns):
    print(col, base_info[col].nunique(dropna=False))

id 24865
oplocdistrict 16
industryphy 20
industryco 346
dom 23278
opscope 20815
enttype 17
enttypeitem 32
opfrom 6620
opto 5747
state 6
orgid 78
jobid 434
adbusign 2
townsign 2
regtype 3
empnum 63
compform 3
parnum 52
exenum 51
opform 34
ptbusscope 1
venind 4
enttypeminu 27
midpreindcode 1
protype 3
oploc 5351
regcap 1144
reccap 598
forreccap 12
forregcap 39
congro 34
enttypegb 53



In [25]:
# 去掉只有一种类别的 columns

del base_info['ptbusscope']
del base_info['midpreindcode']

gc.collect()

1797

In [26]:
# 去掉类别太多的 columns

del base_info['dom']
del base_info['opscope']  # 后面可以分词后用 svd 抽取特征, 但感觉效果不大

gc.collect()

20

In [9]:
base_info.head()

,id,oplocdistrict,industryphy,industryco,enttype,enttypeitem,opfrom,opto,state,orgid,jobid,adbusign,townsign,regtype,empnum,compform,parnum,exenum,opform,venind,enttypeminu,protype,oploc,regcap,reccap,forreccap,forregcap,congro,enttypegb
0,47645761dc56bb8c5fae00114b768b5d9b6e917c3aec07c4,340223,M,7513.00,1100,1150.00,2019-07-11 00:00:00,NaN,6,340223010010000000,340200000000115392,0,0,1,5.00,nan,nan,nan,NaN,nan,1151.00,nan,2367b4cac96d8598,50.00,nan,nan,nan,nan,1151
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,340222,O,8090.00,9600,nan,2017-09-06,NaN,6,340222060010000000,340200000000112114,0,1,1,3.00,1.00,nan,nan,10,3.00,nan,nan,31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3...,10.00,nan,nan,nan,nan,9600
2,59b38c56de3836838082cfcb1a298951abfe15e6940c49ba,340202,R,9053.00,1100,1150.00,2020-09-14 14:46:30,NaN,6,340202010010000000,400000000000753910,0,0,1,2.00,nan,1.00,nan,NaN,nan,1151.00,nan,2367b4cac96d8598,100.00,nan,nan,nan,nan,1151
3,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,340221,L,7212.00,4500,4540.00,2015-09-30,NaN,6,340221010010000000,400000000000013538,0,1,1,2.00,nan,nan,nan,01-以个人财产出资,nan,nan,nan,2367b4cac96d8598,10.00,nan,nan,nan,nan,4540
4,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,340202,R,8810.00,1100,1130.00,2017-12-01,2067-11-30,7,340200000000000000,400000000000283237,0,0,1,nan,nan,nan,nan,NaN,nan,nan,nan,2367b4cac96d8598,100.00,nan,nan,nan,nan,1130


In [44]:
# 时间转换, 暂时先抽取年份特征

base_info['opfrom'] = pd.to_datetime(base_info.opfrom)
base_info['opfrom_year'] = base_info['opfrom'].dt.year.astype('int')
base_info['opto'] = pd.to_datetime(base_info.opto)
base_info['opto_year'] = base_info['opto'].dt.year.fillna(-1).astype('int')

del base_info['opfrom']
del base_info['opto']
gc.collect()

NameError: name 'base_info' is not defined

In [29]:
base_info.head()

,id,oplocdistrict,industryphy,industryco,enttype,enttypeitem,state,orgid,jobid,adbusign,townsign,regtype,empnum,compform,parnum,exenum,opform,venind,enttypeminu,protype,oploc,regcap,reccap,forreccap,forregcap,congro,enttypegb,opfrom_year,opto_year
0,47645761dc56bb8c5fae00114b768b5d9b6e917c3aec07c4,340223,M,7513.00,1100,1150.00,6,340223010010000000,340200000000115392,0,0,1,5.00,nan,nan,nan,NaN,nan,1151.00,nan,2367b4cac96d8598,50.00,nan,nan,nan,nan,1151,2019,-1
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,340222,O,8090.00,9600,nan,6,340222060010000000,340200000000112114,0,1,1,3.00,1.00,nan,nan,10,3.00,nan,nan,31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3...,10.00,nan,nan,nan,nan,9600,2017,-1
2,59b38c56de3836838082cfcb1a298951abfe15e6940c49ba,340202,R,9053.00,1100,1150.00,6,340202010010000000,400000000000753910,0,0,1,2.00,nan,1.00,nan,NaN,nan,1151.00,nan,2367b4cac96d8598,100.00,nan,nan,nan,nan,1151,2020,-1
3,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,340221,L,7212.00,4500,4540.00,6,340221010010000000,400000000000013538,0,1,1,2.00,nan,nan,nan,01-以个人财产出资,nan,nan,nan,2367b4cac96d8598,10.00,nan,nan,nan,nan,4540,2015,-1
4,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,340202,R,8810.00,1100,1130.00,7,340200000000000000,400000000000283237,0,0,1,nan,nan,nan,nan,NaN,nan,nan,nan,2367b4cac96d8598,100.00,nan,nan,nan,nan,1130,2017,2067


In [12]:
base_info.dtypes

id                object
oplocdistrict      int64
industryphy       object
industryco       float64
enttype            int64
enttypeitem      float64
state              int64
orgid              int64
jobid              int64
adbusign           int64
townsign           int64
regtype            int64
empnum           float64
compform         float64
parnum           float64
exenum           float64
opform            object
venind           float64
enttypeminu      float64
protype          float64
oploc             object
regcap           float64
reccap           float64
forreccap        float64
forregcap        float64
congro           float64
enttypegb          int64
opfrom_year        int64
opto_year          int64
dtype: object

In [30]:
# 类别清理

for col in tqdm(['industryco', 'enttypeitem', 'compform', 'venind', 
                 'enttypeminu', 'protype']):
    base_info[col] = base_info[col].fillna(-1).astype('int')

In [14]:
# 数字类型

base_info.select_dtypes(['float64']).describe().T

,count,mean,std,min,25%,50%,75%,max
empnum,19615.00,4.40,15.39,0.00,2.00,3.00,5.00,1500.00
parnum,2339.00,4.59,8.75,0.00,2.00,2.00,3.00,100.00
exenum,1378.00,77.75,2693.74,0.00,1.00,1.00,3.00,100000.00
regcap,24674.00,5151.44,67770.86,0.00,15.00,80.00,500.00,5000100.00
reccap,7084.00,4198.17,36537.98,0.00,0.00,0.00,100.00,1278900.00
forreccap,227.00,292.64,1841.31,0.00,0.00,0.00,0.00,15428.17
forregcap,250.00,1212.58,8113.27,0.00,0.00,0.00,0.00,88817.92
congro,249.00,2805.26,18131.95,0.00,0.00,0.00,0.00,221453.76


In [31]:
# 数字类型缺失值太多, 且部分 columns std 较大, 先考虑用中值填充

for col in tqdm(base_info.select_dtypes(['float64']).columns):
    base_info[col] = base_info[col].fillna(base_info[col].median())

In [32]:
# object 类型

base_info.select_dtypes(['object'])

,id,industryphy,opform,oploc
0,47645761dc56bb8c5fae00114b768b5d9b6e917c3aec07c4,M,NaN,2367b4cac96d8598
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,O,10,31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3...
2,59b38c56de3836838082cfcb1a298951abfe15e6940c49ba,R,NaN,2367b4cac96d8598
3,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,L,01-以个人财产出资,2367b4cac96d8598
4,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,R,NaN,2367b4cac96d8598
...,...,...,...,...
24860,f1c1045b13d18329a2bd99d2a7e2227688c0d69bf1d1e325,O,10,f67c1b92f52ac52e424308ab51241cdef9be3e39c8e1c6...
24861,f000950527a6feb6bde38216d7cbbf32e66d3a3a96d4dbda,J,NaN,2367b4cac96d8598
24862,da8691b210adb3f65b43370d3a362f4aa1d3b16b5ba0c9d7,O,10,2367b4cac96d8598
24863,516ab81418ed215dcbbf0614a7b929e691f8eed153d7bb31,O,NaN,2367b4cac96d8598


In [33]:
# 数据清理
base_info['opform'] = base_info['opform'].replace('01', '01-以个人财产出资').replace('02', '02-以家庭共有财产作为个人出资')

In [35]:
# 数据比较长尾, label encoding 和 freq 处理

for col in tqdm(['industryphy', 'opform', 'oploc', 'orgid', 'jobid', 'oplocdistrict',
                 'enttypegb', 'industryco', 'enttype', 'enttypeitem']):
    lbl = LabelEncoder()
    base_info[col] = lbl.fit_transform(base_info[col].astype(str))
    vc = base_info[col].value_counts(dropna=True, normalize=True).to_dict()
    base_info[f'{col}_freq'] = base_info[col].map(vc)
#     base_info[col] = base_info[col].map(vc)

0         4
1         6
2         9
3         3
4         9
         ..
24860     6
24861    19
24862     6
24863     6
24864     5
Name: industryphy, Length: 24865, dtype: int32
0        26
1        23
2        26
3         1
4        26
         ..
24860    23
24861    26
24862    23
24863    26
24864    23
Name: opform, Length: 24865, dtype: int32
0          91
1        1100
2          91
3          91
4          91
         ... 
24860    4362
24861      91
24862      91
24863      91
24864     964
Name: oploc, Length: 24865, dtype: int32
0        59
1        54
2        26
3        46
4         1
         ..
24860    14
24861     1
24862    41
24863    10
24864    55
Name: orgid, Length: 24865, dtype: int32
0         16
1        392
2        360
3        248
4        275
        ... 
24860    185
24861     19
24862     15
24863    225
24864    131
Name: jobid, Length: 24865, dtype: int32
0         2
1        15
2         9
3        14
4         9
         ..
24860     4
24861    12

In [36]:
# 通过上一次模型的重要度输出发现的分裂次数为 0 的特征
del base_info['forreccap']
del base_info['forregcap']
del base_info['protype']
del base_info['congro']

gc.collect()

1471

In [20]:
base_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24865 entries, 0 to 24864
Data columns (total 35 columns):
id                    24865 non-null object
oplocdistrict         24865 non-null int64
industryphy           24865 non-null int64
industryco            24865 non-null int64
enttype               24865 non-null int64
enttypeitem           24865 non-null int64
state                 24865 non-null int64
orgid                 24865 non-null int64
jobid                 24865 non-null int64
adbusign              24865 non-null int64
townsign              24865 non-null int64
regtype               24865 non-null int64
empnum                24865 non-null float64
compform              24865 non-null int64
parnum                24865 non-null float64
exenum                24865 non-null float64
opform                24865 non-null int64
venind                24865 non-null int64
enttypeminu           24865 non-null int64
oploc                 24865 non-null int64
regcap                24

In [39]:
df_anreport_info = pd.read_csv(t_annual_report_info, header=0)
print(df_anreport_info.head())

for col in tqdm(df_anreport_info.columns):
    print(col, df_anreport_info[col].nunique(dropna=False))

gc.collect()

                                                 id  ANCHEYEAR  STATE  FUNDAM  \
0  9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3    2017.00   2.00    5.00   
1  9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3    2018.00   2.00    2.00   
2  f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb    2017.00   2.00     nan   
3  f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb    2018.00   2.00     nan   
4  9c7fa510616a68309e4badf2a7a3123c0462fb85bf28ef17    2017.00   2.00    5.00   

   MEMNUM  FARNUM  ANNNEWMEMNUM  ANNREDMEMNUM  EMPNUM  EMPNUMSIGN BUSSTNAME  \
0     nan     nan           nan           nan   10.00         nan       NaN   
1     nan     nan           nan           nan    2.00         nan       NaN   
2     nan     nan           nan           nan    4.00        2.00        开业   
3     nan     nan           nan           nan    3.00        2.00        开业   
4     nan     nan           nan           nan   10.00         nan       NaN   

   COLGRANUM  RETSOLNUM  DISPERNUM  UN

id 8937
ANCHEYEAR 4
STATE 3
FUNDAM 168
MEMNUM 6
FARNUM 7
ANNNEWMEMNUM 4
ANNREDMEMNUM 2
EMPNUM 271
EMPNUMSIGN 3
BUSSTNAME 5
COLGRANUM 44
RETSOLNUM 15
DISPERNUM 6
UNENUM 36
COLEMPLNUM 95
RETEMPLNUM 27
DISEMPLNUM 17
UNEEMPLNUM 73
WEBSITSIGN 3
FORINVESTSIGN 3
STOCKTRANSIGN 3
PUBSTATE 4



In [46]:
df_anreport_info.dtypes

id                object
ANCHEYEAR        float64
STATE            float64
FUNDAM           float64
MEMNUM           float64
FARNUM           float64
ANNNEWMEMNUM     float64
ANNREDMEMNUM     float64
EMPNUM           float64
EMPNUMSIGN       float64
COLGRANUM        float64
RETSOLNUM        float64
DISPERNUM        float64
UNENUM           float64
COLEMPLNUM       float64
RETEMPLNUM       float64
DISEMPLNUM       float64
UNEEMPLNUM       float64
WEBSITSIGN       float64
FORINVESTSIGN    float64
STOCKTRANSIGN    float64
PUBSTATE         float64
busstname_停业       uint8
busstname_开业       uint8
busstname_歇业       uint8
busstname_清算       uint8
dtype: object

In [47]:
df_af = pd.get_dummies(df_anreport_info['BUSSTNAME'], prefix='busstname')  
print(df_af)  

KeyError: 'BUSSTNAME'

In [18]:
df_anreport_info = pd.concat([df_anreport_info, df_af], axis=1)
del df_anreport_info['BUSSTNAME']
print(df_anreport_info.dtypes)
print(df_anreport_info.head())


NameError: name 'df_anreport_info' is not defined

In [38]:
df_tax_info = pd.read_csv(t_tax_info, header=0)
print("tax info: \n", df_tax_info)
print("tax info: \n", df_tax_info.shape)

tax info: 
                                                      id  START_DATE  \
0      f000950527a6feb6c2f40c9d8477e73a439dfa0897830397  2015/09/01   
1      f000950527a6feb6c2f40c9d8477e73a439dfa0897830397  2015/09/01   
2      f000950527a6feb6c2f40c9d8477e73a439dfa0897830397  2015/09/01   
3      f000950527a6feb6c2f40c9d8477e73a439dfa0897830397  2015/09/01   
4      f000950527a6feb6c2f40c9d8477e73a439dfa0897830397  2015/09/01   
...                                                 ...         ...   
29190  f000950527a6feb6cb8976eb56233ede461cb23103f85f32  2015/12/08   
29191  f000950527a6feb6cb8976eb56233ede461cb23103f85f32  2015/12/08   
29192  d8071a739aa75a3bbb9e08ebd134ae1289f194b70cac0e95  2016/07/01   
29193  d8071a739aa75a3bbb9e08ebd134ae1289f194b70cac0e95  2016/07/01   
29194  d8071a739aa75a3bbb9e08ebd134ae1289f194b70cac0e95  2016/07/01   

         END_DATE TAX_CATEGORIES     TAX_ITEMS  TAXATION_BASIS  TAX_RATE  \
0      2015/09/30            印花税      工伤保险（单位）        72530

In [39]:
## 提取天数差
from util import *
df_tax_info['TAX_DAYS'] = df_tax_info[['START_DATE', 'END_DATE']].apply(lambda x : days_v1(x['END_DATE'], x['START_DATE']), axis=1)

del df_tax_info['START_DATE']
del df_tax_info['END_DATE']

print(df_tax_info.dtypes)
print(df_tax_info.head())

## float类型的数据转成每天
# df_tax_info['TAXATION_BASIS'] = df_tax_info['TAXATION_BASIS'] / df_tax_info['TAX_DAYS']
# df_tax_info['TAX_RATE'] = df_tax_info['TAX_RATE'] / df_tax_info['TAX_DAYS']
# df_tax_info['DEDUCTION'] = df_tax_info['DEDUCTION'] / df_tax_info['TAX_DAYS']
# df_tax_info['TAX_AMOUNT'] = df_tax_info['TAX_AMOUNT'] / df_tax_info['TAX_DAYS']

# del df_tax_info['TAX_DAYS'] 
print(df_tax_info.head())


id                 object
TAX_CATEGORIES     object
TAX_ITEMS          object
TAXATION_BASIS    float64
TAX_RATE          float64
DEDUCTION         float64
TAX_AMOUNT        float64
TAX_DAYS            int64
dtype: object
                                                 id TAX_CATEGORIES  \
0  f000950527a6feb6c2f40c9d8477e73a439dfa0897830397            印花税   
1  f000950527a6feb6c2f40c9d8477e73a439dfa0897830397            印花税   
2  f000950527a6feb6c2f40c9d8477e73a439dfa0897830397            印花税   
3  f000950527a6feb6c2f40c9d8477e73a439dfa0897830397            印花税   
4  f000950527a6feb6c2f40c9d8477e73a439dfa0897830397            印花税   

      TAX_ITEMS  TAXATION_BASIS  TAX_RATE  DEDUCTION  TAX_AMOUNT  TAX_DAYS  
0      工伤保险（单位）        72530.75      0.00      -0.04       21.80        29  
1      失业保险（单位）        72530.75      0.00      -0.04       21.80        29  
2      医疗保险（单位）        72530.75      0.00      -0.04       21.80        29  
3  企业养老保险基金（单位）        72530.75      0.00      -0

In [40]:
# one-hot
df_tax_item = pd.get_dummies(df_tax_info['TAX_ITEMS'], prefix='tax_items')
df_tax_cate = pd.get_dummies(df_tax_info['TAX_CATEGORIES'], prefix='tax_categories')

df_tax_info = pd.concat([df_tax_info, df_tax_item, df_tax_cate], axis=1)

del df_tax_info['TAX_ITEMS']
del df_tax_info['TAX_CATEGORIES']

print(df_tax_info.head())
print(df_tax_info.shape)


                   0                 0   
3                     0                     0                 0   
4                     0                     0                 0   

   tax_items_失业保险滞纳金  tax_items_失业保险违章罚款  tax_items_失业保险（单位）  \
0                  0                   0                   0   
1                  0                   0                   1   
2                  0                   0                   0   
3                  0                   0                   0   
4                  0                   0                   0   

   tax_items_失地农民养老保险费  tax_items_契税  tax_items_娱乐业  tax_items_安装  \
0                    0             0              0             0   
1                    0             0              0             0   
2                    0             0              0             0   
3                    0             0              0             0   
4                    0             0              0             0   

   tax_items_对企事业单位的承包、

In [42]:
print(df_tax_info.columns)
print("items: \n", df_tax_item.columns)
print("categories: \n", df_tax_cate.columns)

Index(['id', 'TAXATION_BASIS', 'TAX_RATE', 'DEDUCTION', 'TAX_AMOUNT',
       'TAX_DAYS', 'tax_items_10吨以下非机动船', 'tax_items_150吨以下机动船',
       'tax_items_20人以上汽车及2.2升以上小轿车', 'tax_items_20人以下乘人汽车',
       ...
       'tax_categories_城市维护建设税', 'tax_categories_城镇土地使用税', 'tax_categories_契税',
       'tax_categories_房产税', 'tax_categories_教育费附加', 'tax_categories_残疾人就业保障金',
       'tax_categories_水利建设专项收入', 'tax_categories_税务部门罚没收入',
       'tax_categories_耕地占用税', 'tax_categories_营业税'],
      dtype='object', length=298)
items: 
 Index(['tax_items_10吨以下非机动船', 'tax_items_150吨以下机动船',
       'tax_items_20人以上汽车及2.2升以上小轿车', 'tax_items_20人以下乘人汽车', 'tax_items_一等',
       'tax_items_一等（县区）', 'tax_items_一类地区', 'tax_items_一类地区（占用基本农田）',
       'tax_items_一般房屋出租', 'tax_items_三等',
       ...
       'tax_items_销售建筑物或构筑物', 'tax_items_销售额或营业额', 'tax_items_非普通住宅（预征）',
       'tax_items_非机动车', 'tax_items_面积（旅店业、餐饮业、娱乐业）', 'tax_items_预提所得税',
       'tax_items_飞机场跑道、停机坪', 'tax_items_餐饮、住宿、娱乐营业额随征', 'tax_items_餐饮业',

In [62]:
items_nums = len(df_tax_item.columns)
categories_nums = len(df_tax_cate.columns)
# print("items nums : [%d]" % items_nums)
# print("categories nums : [%d]" % categories_nums)

# TAX_ITEMS  TAXATION_BASIS  TAX_RATE  DEDUCTION  TAX_AMOUNT  TAX_DAYS 

for id, col in enumerate(df_tax_item.columns):
    df_tmp = df_tax_info.groupby(['id',col]).sum()
    df_to_merge = df_tmp[['id', 'TAXATION_BASIS', 'TAX_RATE', 'DEDUCTION', 'TAX_AMOUNT', 'TAX_DAYS']]
    if(id == 0):
        print(df_tmp.head())
    # print(df_tmp)
    # if(id == 0):
    #     df_tax = df_tmp
    # df_tax = pd.concat([df_tax, df_tmp], axis=1)

# print(df_tax.shape)

KeyError: "['id'] not in index"

In [31]:
df_categories = df_tax_info.groupby(['id','TAX_CATEGORIES'], as_index=False)
print(df_categories.head())


                                                     id  START_DATE  \
0      f000950527a6feb6c2f40c9d8477e73a439dfa0897830397  2015/09/01   
1      f000950527a6feb6c2f40c9d8477e73a439dfa0897830397  2015/09/01   
2      f000950527a6feb6c2f40c9d8477e73a439dfa0897830397  2015/09/01   
3      f000950527a6feb6c2f40c9d8477e73a439dfa0897830397  2015/09/01   
4      f000950527a6feb6c2f40c9d8477e73a439dfa0897830397  2015/09/01   
...                                                 ...         ...   
29166  f000950527a6feb6cb8976eb56233ede461cb23103f85f32  2015/12/08   
29173  f000950527a6feb6cb8976eb56233ede461cb23103f85f32  2015/12/08   
29192  d8071a739aa75a3bbb9e08ebd134ae1289f194b70cac0e95  2016/07/01   
29193  d8071a739aa75a3bbb9e08ebd134ae1289f194b70cac0e95  2016/07/01   
29194  d8071a739aa75a3bbb9e08ebd134ae1289f194b70cac0e95  2016/07/01   

         END_DATE TAX_CATEGORIES     TAX_ITEMS  TAXATION_BASIS  TAX_RATE  \
0      2015/09/30            印花税      工伤保险（单位）        72530.75      0.0

In [13]:
for col in tqdm(df_tax_info.columns):
    print(col, df_tax_info[col].nunique(dropna=False))

id 808
START_DATE 91
END_DATE 91
TAX_CATEGORIES 17
TAX_ITEMS 275
TAXATION_BASIS 1666
TAX_RATE 29
DEDUCTION 248
TAX_AMOUNT 4568

